Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm(A::Matrix{T},f::Fun) where {T<:Number}
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π;a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ Operator(diagm(fill(ldirichlet(d),n)));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

5.551115123125783e-16 - 0.2683546905319053im
4.657190431911484e-16 + 0.2683546905319056im
-0.03747531973228899 + 0.9999999999999997im
0.03747531973228914 + 0.9999999999999992im

Floquet Multipliers = 

0.6651802570066617 - 0.7466828146465908im
0.6651802570066608 + 0.7466828146465911im
-0.8889340869219562 + 8.983951009601116e-16im
-1.1249427991479366 + 2.6356405538206627e-15im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(xsol)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1 
 
 
 0 
 
 
 1 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 592.126,281.192 592.122,281.233 592.113,281.314 592.099,281.437 592.082,281.6 592.06,281.803 592.033,282.048 592.002,282.332 591.967,282.657 591.927,283.021 
 591.883,283.426 591.834,283.87 591.781,284.353 591.723,284.876 591.661,285.437 591.595,286.037 591.524,286.674 591.449,287.35 591.369,288.062 591.285,288.812 
 591.197,289.598 591.104,290.419 591.007,291.277 590.905,292.168 590.799,293.095 590.689,294.054 590.574,295.047 590.454,296.072 590.331,297.129 590.203,298.216 
 590.07,299.333 589.933,300.48 589.792,301.655 589.646,302.857 589.496,304.086 589.342,305.341 589.183,306.62 589.02,307.922 588.852,309.247 588.68,310.593 
 588.504,311.959 588.323,313.344 588.138,314.746 587.949,316.165 587.755,317.599 587.557,319.046 587.355,320.505 587.148,321.975 586.937,323.454 586.721,324.941 
 586.501,326.433 586.277,327.93 586.049,329.429 585.816,330.929 585.579,332.429 585.337,333.925 585.091,335.417 584.841,336.902 584.587,338.379 584.328,339.845 
 584.065,341.299 583.798,342.739 583.526,344.162 583.25,345.566 582.97,346.95 582.685,348.31 582.397,349.646 582.103,350.954 581.806,352.233 581.505,353.479 
 581.199,354.692 580.889,355.868 580.574,357.006 580.256,358.103 579.933,359.156 579.606,360.164 579.275,361.124 578.939,362.034 578.6,362.891 578.256,363.694 
 577.908,364.439 577.555,365.125 577.199,365.75 576.838,366.31 576.473,366.805 576.104,367.231 575.731,367.587 575.354,367.87 574.972,368.078 574.586,368.209 
 574.197,368.262 573.803,368.233 573.405,368.121 573.002,367.924 572.596,367.64 572.185,367.268 571.771,366.804 571.352,366.248 570.929,365.597 570.503,364.85 
 570.072,364.005 569.637,363.06 569.198,362.014 568.754,360.865 568.307,359.612 567.856,358.252 567.401,356.784 566.941,355.208 566.478,353.52 566.011,351.721 
 565.539,349.807 565.064,347.78 564.585,345.635 564.101,343.374 563.614,340.994 563.123,338.494 562.628,335.873 562.128,333.13 561.625,330.264 561.118,327.275 
 560.607,324.16 560.092,320.92 559.574,317.554 559.051,314.061 558.524,310.441 557.994,306.693 557.459,302.818 556.921,298.816 556.379,294.685 555.833,290.428 
 555.283,286.044 554.729,281.535 554.172,276.901 553.61,272.143 553.045,267.264 552.476,262.264 551.904,257.147 551.327,251.915 550.747,246.571 550.163,241.118 
 549.575,235.559 548.984,229.9 548.388,224.144 547.789,218.296 547.187,212.362 546.58,206.348 545.97,200.261 545.356,194.106 544.739,187.893 544.118,181.627 
 543.493,175.318 542.864,168.975 542.232,162.607 541.596,156.223 540.957,149.834 540.314,143.45 539.668,137.083 539.017,130.742 538.364,124.441 537.706,118.19 
 537.046,112.002 536.381,105.889 535.713,99.8628 535.042,93.9364 534.367,88.1221 533.689,82.4321 533.007,76.8787 532.321,71.4742 531.632,66.2303 530.94,61.1588 
 530.244,56.2711 529.545,51.5781 528.842,47.0907 528.136,42.8189 527.427,38.7724 526.714,34.9606 525.998,31.3921 525.278,28.075 524.555,25.0169 523.829,22.2248 
 523.099,19.7051 522.366,17.4635 521.63,15.5053 520.89,13.8352 520.148,12.457 519.401,11.3745 518.652,10.5906 517.899,10.1077 517.144,9.92788 516.384,10.0527 
 515.622,10.4834 514.857,11.2205 514.088,12.2646 513.316,13.6155 512.541,15.2731 511.763,17.2365 510.982,19.5051 510.197,22.0773 509.409,24.9518 508.619,28.1266 
 507.825,31.5996 507.028,35.3681 506.228,39.4293 505.425,43.7798 504.619,48.4157 503.81,53.3328 502.998,58.5261 502.183,63.99 501.365,69.7184 500.544,75.7041 
 499.72,81.9395 498.893,88.4157 498.064,95.1231 497.231,102.051 496.395,109.187 495.557,116.519 494.715,124.033 493.871,131.713 493.024,139.543 492.174,147.505 
 491.321,155.58 490.465,163.749 489.607,171.989 488.746,180.28 487.882,188.598 487.015,196.918 486.146,205.218 485.273,213.471 484.399,221.652 483.521,229.736 
 482.641,237.696 481.758,245.507 480.872,253.145 479.